# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv(".\Weatherpy Documents\weather_df.csv")
weather_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.12,-134.97,75.99,83,81,17.27,PF,1605389950
1,1,Busselton,-33.65,115.33,48.00,98,3,13.47,AU,1605389951
2,2,Nikolskoye,59.70,30.79,32.00,86,75,4.47,RU,1605389954
3,3,Puerto Escondido,15.85,-97.07,84.20,70,20,7.09,MX,1605389956
4,4,Ushuaia,-54.80,-68.30,44.60,65,90,6.93,AR,1605389937
...,...,...,...,...,...,...,...,...,...,...
559,559,Plettenberg Bay,-34.05,23.37,62.69,85,94,6.71,ZA,1605390799
560,560,Kuantan,3.81,103.33,77.00,94,20,3.36,MY,1605390800
561,561,Amapá,1.00,-52.00,74.89,93,100,2.84,BR,1605390802
562,562,Iskateley,67.68,53.15,28.47,98,98,13.15,RU,1605390803


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Adjusted temp range to 75-80 to get to a list of 10 cities only.

vacation_df = weather_df[(weather_df["Max Temperature"] > 75) & (weather_df["Max Temperature"] < 80) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)]

vacation_df = vacation_df.reset_index()

vacation_df

,index,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,79,79,Port Hedland,-20.32,118.57,75.20,69,0,8.05,AU,1605390067
1,140,140,Yanam,16.73,82.22,78.82,74,0,9.31,IN,1605390158
2,162,162,Makkah al Mukarramah,21.43,39.83,76.05,73,0,3.83,SA,1605390121
3,229,229,Hambantota,6.12,81.12,77.00,100,0,3.36,LK,1605390296
4,286,286,Dwārka,22.24,68.97,77.92,61,0,7.87,IN,1605390381
5,290,290,Oussouye,12.48,-16.55,78.80,74,0,4.70,SN,1605390386
6,295,295,Nazca,-14.83,-74.95,76.82,40,0,7.49,PE,1605390395
7,327,327,Poum,-20.23,164.02,78.28,80,0,7.52,NC,1605390446
8,331,331,Los Llanos de Aridane,28.66,-17.92,75.20,64,0,5.82,ES,1605390451
9,376,376,Abu Dhabi,24.47,54.37,78.80,54,0,6.93,AE,1605390519


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_df[["City", "Latitude","Longitude","Max Temperature","Humidity","Cloudiness","Wind Speed","Country","Date"]]
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Hedland,-20.32,118.57,75.20,69,0,8.05,AU,1605390067
1,Yanam,16.73,82.22,78.82,74,0,9.31,IN,1605390158
2,Makkah al Mukarramah,21.43,39.83,76.05,73,0,3.83,SA,1605390121
3,Hambantota,6.12,81.12,77.00,100,0,3.36,LK,1605390296
4,Dwārka,22.24,68.97,77.92,61,0,7.87,IN,1605390381
5,Oussouye,12.48,-16.55,78.80,74,0,4.70,SN,1605390386
6,Nazca,-14.83,-74.95,76.82,40,0,7.49,PE,1605390395
7,Poum,-20.23,164.02,78.28,80,0,7.52,NC,1605390446
8,Los Llanos de Aridane,28.66,-17.92,75.20,64,0,5.82,ES,1605390451
9,Abu Dhabi,24.47,54.37,78.80,54,0,6.93,AE,1605390519


In [7]:

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"


In [8]:
hotel_name = []

for row in range(len(hotel_df["City"])):

    lat = hotel_df.loc[row]["Latitude"]
    lon = hotel_df.loc[row]["Longitude"]
    
    query_url = f"{url}location={lat},{lon}&radius=5000&keyword=hotel&key={g_key}"

    google_response = requests.get(query_url)
    place_json = google_response.json()
    
    try: 
        hotel_name.append(place_json["results"][0]["name"])
        
    except:
        hotel_name.append("Hotel not found")

print(hotel_name)

['Hedland Hotel', 'The Regency Hotel', 'رمادا زاد القصر', 'The Peacock Beach Hotel Hambantota', 'Hawthorn Suites by Wyndham Dwarka', 'Campement ALULUM', 'Hotel Hacienda Majoro', 'Hotel not found', 'Hotel Hacienda de Abajo', 'Villaggio Hotel & Resort']


In [9]:
hotel_df["Hotel"] = hotel_name
hotel_df

C:\Users\kylie\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,Port Hedland,-20.32,118.57,75.20,69,0,8.05,AU,1605390067,Hedland Hotel
1,Yanam,16.73,82.22,78.82,74,0,9.31,IN,1605390158,The Regency Hotel
2,Makkah al Mukarramah,21.43,39.83,76.05,73,0,3.83,SA,1605390121,رمادا زاد القصر
3,Hambantota,6.12,81.12,77.00,100,0,3.36,LK,1605390296,The Peacock Beach Hotel Hambantota
4,Dwārka,22.24,68.97,77.92,61,0,7.87,IN,1605390381,Hawthorn Suites by Wyndham Dwarka
5,Oussouye,12.48,-16.55,78.80,74,0,4.70,SN,1605390386,Campement ALULUM
6,Nazca,-14.83,-74.95,76.82,40,0,7.49,PE,1605390395,Hotel Hacienda Majoro
7,Poum,-20.23,164.02,78.28,80,0,7.52,NC,1605390446,Hotel not found
8,Los Llanos de Aridane,28.66,-17.92,75.20,64,0,5.82,ES,1605390451,Hotel Hacienda de Abajo
9,Abu Dhabi,24.47,54.37,78.80,54,0,6.93,AE,1605390519,Villaggio Hotel & Resort


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))